# Predictive Regression

## Imports

In [207]:
# <include-predictive_regression/utils.py>

In [208]:
# <imports>
import numpy as np
import pandas as pd
import plotly.io as pio

from predictive_regression import utils

pd.options.plotting.backend = "plotly"
pio.templates.default = "seaborn"
# pio.base_renderers.default = "vscode"

from patsy import dmatrices
import statsmodels.api as sm

## Summary

In this assignment we create a two-stage regression model to predict the future weekly "returns" of five-year credit default swaps for 12 publicly traded, large-capitalization companies. The first stage entails creating contemporaneous models of the CDS "return" and stock price return and then calculating residuals. In the second stage, we create a model that predicts the CDS "return" residual based on the prior observation's equity return residual. Our period of analysis spans from 2018-01-03 to 2021-4-30.

The real world scenario here can be described as predicting a change in bond prices based on the prior oberservation's change in equity price. One thing that may limit the predictive power of our models is that we are using a full week as our period of analysis. Unless the timing of the change in equity price is such that it occurrs at the end of a week, such that if there is a lag to the adjustment of the bond price, it is likely to occurr in the subsequent period, our model won't pick it up if it occurs in the same period as the equity price adjustment.

### Contemporaneous Models

We model contemporaneous CDS "returns" as a function of stock price return and "return" an index of other CDS "returns", defined as the arithmetic average the CDS's of the other 11 companies in our universe. For each of these models we perform ordinary least squares regression for each ticker, for each contiguous 16 week period throughout our entire period of analysis. 

$$
r_{E}^{CDS} \sim r_{E}^{Equity} + r_{Index}^{CDS} + \epsilon
$$

We model contemporaneous equity returns as a function of the return on the market, defined as the return on SPY.

$$
r_{E}^{Equity} \sim r_{SPY}^{Equity} + \epsilon
$$

To predict the contemporaneous CDS return we end up with
$$
f_{E,n} = \beta_{E,n}^{Intercept} + \beta_{E,n}^{Equity} \cdot r_{E,n}^{Equity} + \beta_{E,n}^{Index} \cdot r_{E,n}^{Index}
$$

$$
g_{E,n} = \gamma_{E,n}^{Intercept} + \gamma_{E,n}^{SPY} \cdot r_{SPY,n}^{Equity}
$$

Contemporanous residuals can the be defined as for CDS
$$
\rho_{E,n} = f_{E,n} - r_{E,n}^{CDS} 
$$

and

$$
c_{E,n} = g_{E,n} - r_{E,n}^{Equity} 
$$

### Predictive Model

Our predictive model then becomes

$$
\rho_{E,n} = c_{E,n-1} + \epsilon
$$

where we are lagging the equity residual back one observation from the CDS residual and using the various window methodologies and lengths described above to calculate the regression coefficients. Our model then becomes

$$
h_{E,n} = \mu_{E,n}^{Intercept} + \mu_{E,n} \cdot c_{E,n-1}
$$

and residuals can be defined as

$$
q_{E,n} = \rho_{E,n} - h_{E,n}
$$

### Analysis
Our job is to compare the performance of the predictive regressions from the exponentially weighted windows to those of the boxcar windows. We first compare the overall r-squared across all tickers for each window methodology and length. However, in the real world, it may be the case that certain combinations of window length and methodology may perform better for certain tickers. To that end, we analyze the r-squared for each ticker for each methodlogy and window length to identify the combination that performs best for each ticker.

Importantly, as is highlighted in the first set of asset performance charts below, the world changed at the outset of the pandemic and developing models that fit well across any period including the pandemic is going to be challenging. In order to get sense for how much more we can improve our model, we construct an alternate set of regression coefficients stopping just before the pandemic and compare their peformance to the models developed from the entire time period.

As mentioned above, we try to improve the performance of our models by reducing the period of analyisis from a week to a day, to see if the shorter duration results in a stronger evidence of a lagged relationship between changes in equity prices and changes in CDS spreads.

### Results

* The best combination of window methodology and length turned out to be a boxcar window with a length of 24 weeks and resulted in an overall $R^{2}$ of 0.093.

* By pulling out the best window methodology-length combination for each ticker were able to increase our overall $R^2$ to 0.131.

## Analysis

We start by fetching the CDS and equity return data.

In [209]:
date_range = pd.date_range("2018-01-03", "2021-04-30", freq="7D")
df_data = utils.get_data(date_range)
df_data.tail()

series              adj_close  r_equity  r_spread  spread5y   r_index  \
date       ticker                                                       
2021-04-28 LUV      61.770000 -0.004362 -0.017298  0.008339 -0.010192   
           MAR     149.450000  0.031885 -0.008381  0.008758 -0.011003   
           T        30.960000  0.027839 -0.023042  0.006872 -0.009670   
           WFC      44.983531  0.041673 -0.031137  0.005364 -0.008934   
           XOM      58.110000  0.036986  0.009787  0.003850 -0.012655   

series                r_spy  
date       ticker            
2021-04-28 LUV     0.003191  
           MAR     0.003191  
           T       0.003191  
           WFC     0.003191  
           XOM     0.003191

Let's take a look at the returns that we ultimately are going to be predicting and their underlying levels to see how tough our job is going to be. This is again, our familiar pattern with conspicuous precipitous decline in asset values at the onset of the pandemic, followed by a period of increased volatiilty and then a resumption of more "normal" behavior in recent times.

In [210]:
utils.make_lines_chart(df_data.spread5y)

In [211]:
utils.make_lines_chart(
    df_data.r_spread.groupby("ticker").cumsum(),
    'Cumulative Weekly "Return" on 5-Year CDS Spread'
)

In [212]:
utils.make_lines_chart(
    df_data.r_equity.groupby("ticker").cumsum(),
    'Cumulative Weekly Equity Return'
)

As a first pass, let's take a look at the correlation of weekly equity price returns and returns on CDS spreads at lags of 0, one and two weeks. This shows that the corelation in the same week is over 50%, but a good portion of that is likely forward looking. Interestingly, after that, at lags from one to four weeks, the correlation is quite similar. That could indicate that the lag relationship is most often ocurring within the week as discussed above, or there is a more muted lag effect that can play out over a longer period of time. Based on the apparent similarity of movements in the two charts above, my initial hypothesis is that the prices actually adjust more quickly and that by using the relatively long period of a week, we are missing it.

In [213]:
corrs = [df_data.r_spread.corr(df_data.r_equity.shift(w)) for w in range(10)]
utils.px.bar(
    x=list(range(10)),
    y=corrs,
    title="Correlation of Equity Returns with Changes in CDS Spread by Lag Weeks",
    labels=dict(y="correlation", x="weeks lag")
)


There are undoudtedly many instances where the changes happen on the same day and simply creating models with no lag with a full week as the period of analysis would be using future data to make predictions of the future. However, it would be interesting to see if the correlation increases if we decreas the period of analysis from a week to a day.

Here we do see that the correlation with a one day period as oppossed to a one week period does have a higher correlation (0.3962 vs 0.3461).

In [214]:
df_daily_data = utils.get_daily_data()
corrs = [df_daily_data.r_spread.corr(df_daily_data.r_equity.shift(w)) for w in range(10)]
utils.px.bar(
    x=list(range(10)),
    y=corrs,
    title="Correlation of Equity Returns with Changes in CDS Spread by Lag Day",
    labels=dict(y="correlation", x="weeks lag")
)


### Contemporaneous Residuals

Here we perform our contemporaneous regressions and calculate residuals for both equity and cds returns. Both use boxcar windows of 16 weeks, but a slightly higher total $R^{2}$ was achieved by running the cds model as a robust linear model using Tukey Bisquare weights.

In [215]:
df_resid = utils.get_contemp_resids(df_data)
df_resid.head()

/home/caleb/.local/share/virtualenvs/predictive_regression-7OMi-ASR/lib/python3.8/site-packages/scipy/stats/stats.py:1603: UserWarning:

kurtosistest only valid for n>=20 ... continuing anyway, n=16



cds_resid  eq_resid
date       ticker                     
2018-05-09 BA      -0.041016  0.064893
           C        0.035478  0.015434
           DD      -0.016231 -0.006366
           F       -0.067737 -0.015277
           GE      -0.052186 -0.024766

A quick look at the moments for each set of residuals reveals the following characteristics:

* Extreme outliers evident in the distribution of cds returns - kurtosis of 32.0 and extremely fat tails in the q/q plot.
* Equity return outliers are high as well, kurtosis of 7.2 and fat tails in the q/q plot, but not as extreme as cd returns
* Interesting to note in the moments that the contemporaneous equity model seems to consistenly over predict equity return, both on a mean and median basis, and the contemporaneous cds return model is the opposite, consistently under-predicting on both a mean and median basis.

In [216]:
df_stats = df_resid.describe()
df_stats.loc["kurtosis"] = df_resid.kurtosis()
df_stats.loc["skewness"] = df_resid.skew()
df_stats

,cds_resid,eq_resid
count,1872.000000,1872.000000
mean,-0.001374,0.000793
std,0.085407,0.046581
min,-1.015630,-0.365794
25%,-0.026964,-0.018887
50%,-0.000057,0.000882
75%,0.024526,0.022092
max,0.954388,0.299017
kurtosis,31.960394,7.192748
skewness,-0.938949,0.092354


In [217]:
fig = utils.make_overview_chart(df_resid.cds_resid, title="CDS Returns Residual")
fig.show()

In [218]:
fig = utils.make_overview_chart(df_resid.eq_resid, title="Equity Returns Residual")
fig.show()

In [219]:
df_resid.cds_resid.corr(df_resid.eq_resid.shift())

-0.02479475475232577

Here we plot the cds residuals agains the prior week's equity residuals and see very little relationship. The fitted line is a simple OLS regression based on the single trailing equity return. The correlation is 0.0987 with an $R^{2}$ of just 0.005. it will interesting to see if this can be improved using discounted least squares, effectively taking into account a greater number of historical observations.

In [220]:
utils.make_residual_scatter(df_resid.copy())

### Predictive Residuals

Here we calculate coefficients for each equity for each observation for each window type for each window length. We do this by using the rolling covariance matrix function in pandas to get the covariance and variance for each ticker and then calculate coefficients directly as $\mu_{E,n} = \frac{\mathrm{Cov}(c_{E,n-1},\rho_{E,n})}{\mathrm{Var}(c_{E,n-1})}$.

In [221]:
 df_resid_pred = utils.get_predicted_resids(df_resid)

Finally, we calculate the total $R^{2}$ for each combination of window type and window length. Interstingly the boxcar windows type with a length between 24 and 48 weeks appears to have the best fit (boxcar window lengths are equal to 2 x the exponentially weighted $1/\lambda$).

In [222]:
fig = utils.make_rsq_comparison(df_resid_pred)
fig.show()

### Predictive Residuals by Ticker

Below we select the highest $R^2$ combination of window type and window length for each ticker. There are actually differences in the fits between the tickers. BA, for example, appears to do relatively better with the exponentially weighted windows. It's max $R^2$ is 0.10 with the exponentially weighted window with $t=12$. MAR, on the other hand, seems to do best with the boxcar window of 12 days (t=6) with an $R^2$ of 0.234.

In [223]:
df_ticker_rsq = utils.get_ticker_rsq(df_resid_pred)

In [224]:
fig = utils.make_rsq_ticker_comparison(df_ticker_rsq)
fig.show()

Here we pull out the best r-sqaured for each ticker. It looks like most are of the boxcar variety and and in the range of 24 to 48 weeks.

In [225]:
max_rsqs = []
for s in df_ticker_rsq.columns.levels[0]:
    w = df_ticker_rsq[s].max().idxmax()
    t = df_ticker_rsq[s][w].idxmax()

    max_rsqs.append((s, w, t, df_ticker_rsq[s][w].max()))


In [226]:
df_max_rsq = pd.DataFrame(max_rsqs)
df_max_rsq.columns = ["ticker", "win_type", "win_length", "r-sq."]
df_max_rsq

,ticker,win_type,win_length,r-sq.
0,BA,exp_wm,t_10,0.240549
1,C,exp_wm,t_46,-0.005902
2,DD,boxcar,t_40,0.010604
3,F,exp_wm,t_08,0.301011
4,GE,boxcar,t_38,-0.004328
5,JPM,exp_wm,t_46,0.039032
6,LOW,boxcar,t_10,0.107542
7,LUV,boxcar,t_06,0.240101
8,MAR,boxcar,t_30,0.097636
9,T,boxcar,t_36,-0.016100


For our final analysis, let's see how much we can improve the overall $R^2$ by using the best window specificication for each ticker. And we see that we are able to increase our overall $R^2$ from 0.093 to 0.131.

In [227]:
best_resids = []
for best in df_max_rsq.iterrows():
    best_resids.append(df_resid_pred.loc[df_resid_pred.index.get_level_values("ticker") == best[1].ticker][[(best[1].win_type, "resid_sq", best[1].win_length), (best[1].win_type, "error_sq", best[1].win_length)]].unstack("ticker"))

In [228]:
df_best_rsq = pd.concat(best_resids, axis=1).sort_index(axis=1).stack("stat").reset_index().groupby("stat").sum().sum(axis=1)
1 - df_best_rsq.resid_sq / df_best_rsq.error_sq

/home/caleb/.local/share/virtualenvs/predictive_regression-7OMi-ASR/lib/python3.8/site-packages/pandas/core/generic.py:4153: PerformanceWarning:

dropping on a non-lexsorted multi-index without a level parameter may impact performance.



0.13091761528714907